- TF-IDF : 텍스트 벡터화
- PCA : 차원축소
- LSA : 잠재 의미 분석
- t-SNE : 2D 시각화
- 로지스틱회귀
- 토큰화&전처리

In [ ]:
# LSA
# TF - IDF 행렬에 대해서 SVD
# 단어와 문서 간의 숨겨진 의미 관계를 찾음
# PCA 차이:
    # PCA : 데이터 자체 분산 최대화
    # LSA : 문서-단어형태의 의미구조 파악
# 은행
    # '돈' '계좌'  주변에 등장
    # '나무' '냄새' '먹는다' 주변에 등장

In [ ]:
# t-SNE : 고차원 데이터를 2D,3D로 변환 - 시각화 전용(분석에는 부적합), 계산이 오래걸림
# PCA VS t-SNE
# PCA : 속도가 빠름, 전역 구조 보존
# t-SNE : 느림, 국소(지역)군집 명확

In [1]:

# 데이터셋
from sklearn.datasets import load_files
train_path = r'C:\python_src\LLM\20newsbydate\20news-bydate-train'
test_path = r'C:\python_src\LLM\20newsbydate\20news-bydate-test'
newsgroups_train = load_files(train_path,encoding='latin1')
newsgroups_test = load_files(test_path,encoding='latin1')
categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].*', '', text)

    return text
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories  ]
    #필터링
    data_filtered, target_filtered = [], []
    for text,label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재 정렬
            data_filtered.append(text) ; target_filtered.append( new_label  )
    return data_filtered,target_filtered,categories
train_data, train_target, target_names = filter_categories(newsgroups_train,categories)
test_data, test_target, _ = filter_categories(newsgroups_test,categories)

x_train = [ clean_text(t) for t in train_data]
x_test = [ clean_text(t) for t in test_data]
y_train = train_target
y_test = test_target

In [ ]:
# 텍스트 전처리 (토큰화(3글자이상) + 불용어제어(Stop words) + 어간추출(stemming)) --> 영어
# 파이프라인
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from TTfidfVectorizer
RegexpTokenizer(r"[\w']{3,}")  # 3글자 이상 단어 추출
english_stopws = set(stopwords.words('english'))
# 커스텀 토크나이저
def tokenizer(text):
    tokens = regtok.tokenize(text)
    words = [word for word in tokens if word not in english_stopws]
    features = list(map(lambda x:PorterStemmer().stem(x), words))
    return features
#TF-IDF 벡터화
tfidf = TfidfVectorizer( tokenizer=tokenizer, max_df=0.5, min_df=2)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
# 분류모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=200,



